In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout

# !pip install fastparquet


In [ ]:
# Read the processed dataset.
semg_df = pd.read_parquet('/content/drive/MyDrive/sEMG-HAR/semg_extracted_scalar_features_only', engine='fastparquet')

semg_df[:20]

In [ ]:
# Separate features and labels

# print(semg_df.columns)

X = semg_df.drop(columns=['activity']).astype('float32')
y = semg_df['activity']

# Normalize features
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# One-hot encoding for the labels
y = to_categorical(y-1, num_classes=21)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)

# Reshape input to be [samples, time steps, features] which is required for LSTM
X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))


# Simple LSTM model

In [ ]:

# LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))  # 50 units in the LSTM layer

model.add(Dropout(0.5))  # dropout layer with dropout rate of 0.5

model.add(Dense(21, activation='softmax'))  # 21 units in the output layer for 21 classes

# Compile and fit the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color=color)
line1, = ax1.plot(history.history['loss'], color=color, label='Train Loss')
line2, = ax1.plot(history.history['val_loss'], color=color, linestyle='dashed', label='Val Loss')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Accuracy', color=color)
line3, = ax2.plot(history.history['accuracy'], color=color, label='Train Acc')
line4, = ax2.plot(history.history['val_accuracy'], color=color, linestyle='dashed', label='Val Acc')
ax2.tick_params(axis='y', labelcolor=color)

# Added this line
lns = [line1, line2, line3, line4]
ax1.legend(handles=lns, loc='center right')

fig.tight_layout()
plt.show()


Stacked LSTM

In [ ]:
#Stacked LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.5))
model.add(LSTM(50))
# model.add(Dropout(0.5))
model.add(Dense(21, activation='softmax'))

# Compile and fit the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
X_train.shape

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color=color)
line1, = ax1.plot(history.history['loss'], color=color, label='Train Loss')
line2, = ax1.plot(history.history['val_loss'], color=color, linestyle='dashed', label='Val Loss')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Accuracy', color=color)
line3, = ax2.plot(history.history['accuracy'], color=color, label='Train Acc')
line4, = ax2.plot(history.history['val_accuracy'], color=color, linestyle='dashed', label='Val Acc')
ax2.tick_params(axis='y', labelcolor=color)

# Added this line
lns = [line1, line2, line3, line4]
ax1.legend(handles=lns, loc='center right')

fig.tight_layout()
plt.show()


Bidirectional LSTM

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(42), input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.6))
model.add(Dense(21, activation='softmax'))

# Compile and fit the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=350, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color=color)
line1, = ax1.plot(history.history['loss'], color=color, label='Train Loss')
line2, = ax1.plot(history.history['val_loss'], color=color, linestyle='dashed', label='Val Loss')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Accuracy', color=color)
line3, = ax2.plot(history.history['accuracy'], color=color, label='Train Acc')
line4, = ax2.plot(history.history['val_accuracy'], color=color, linestyle='dashed', label='Val Acc')
ax2.tick_params(axis='y', labelcolor=color)

# Added this line
lns = [line1, line2, line3, line4]
ax1.legend(handles=lns, loc='center right')

fig.tight_layout()
plt.show()


MLP model

In [ ]:
# Separate features and labels

# print(semg_df.columns)

X = semg_df.drop(columns=['activity']).astype('float32')
y = semg_df['activity']

# Normalize features
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# One-hot encoding for the labels
y = to_categorical(y-1, num_classes=21)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)

# # Reshape input to be [samples, time steps, features] which is required for LSTM
# X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
# X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))


In [ ]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

# MLP model with dropout and L2 regularization
model = Sequential()
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01), input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(21, activation='softmax'))  # 21 units in the output layer for 21 classes

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=100)

# Compile and fit the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=750, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])


In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color=color)
line1, = ax1.plot(history.history['loss'], color=color, label='Train Loss')
line2, = ax1.plot(history.history['val_loss'], color=color, linestyle='dashed', label='Val Loss')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Accuracy', color=color)
line3, = ax2.plot(history.history['accuracy'], color=color, label='Train Acc')
line4, = ax2.plot(history.history['val_accuracy'], color=color, linestyle='dashed', label='Val Acc')
ax2.tick_params(axis='y', labelcolor=color)

# Added this line
lns = [line1, line2, line3, line4]
ax1.legend(handles=lns, loc='center right')

fig.tight_layout()
plt.show()


FNN Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

import numpy as np

# # Split the data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


print("number of columns:", len(X_train.columns))

# Build the FNN model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(42, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(42, activation='relu', kernel_regularizer=l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(21, activation='softmax')
])

initial_learning_rate = 0.01
epochs = 350
batch_size = 32

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")


In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color=color)
line1, = ax1.plot(history.history['loss'], color=color, label='Train Loss')
line2, = ax1.plot(history.history['val_loss'], color=color, linestyle='dashed', label='Val Loss')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Accuracy', color=color)
line3, = ax2.plot(history.history['accuracy'], color=color, label='Train Acc')
line4, = ax2.plot(history.history['val_accuracy'], color=color, linestyle='dashed', label='Val Acc')
ax2.tick_params(axis='y', labelcolor=color)

# Added this line
lns = [line1, line2, line3, line4]
ax1.legend(handles=lns, loc='center right')

fig.tight_layout()
plt.show()


Simple MLP

In [ ]:
import numpy as np

# Assuming you have X_train and y_train as your training data and labels

# Reshape the time domain features to be treated as 2D images
# This assumes X_train has shape: (num_samples, num_features_per_sample)
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[2])

# Create a function to apply data augmentation manually
def augment_data(X, y, num_augmentations=5):
    num_samples, num_features = X.shape
    augmented_X = []
    augmented_y = []
    for i in range(num_samples):
        # Original sample
        augmented_X.append(X[i])
        augmented_y.append(y[i])

        # Data augmentation: Apply random rotations, shifts, or other transformations here
        for _ in range(num_augmentations):
            # For example, you can apply random rotations of +/- 20 degrees
            augmented_X.append(np.rot90(X[i].reshape(1, -1), k=np.random.choice([1, 2, 3]), axes=(1, 0)).flatten())
            augmented_y.append(y[i])

            # Add more data augmentation techniques here, if needed

    return np.array(augmented_X), np.array(augmented_y)

# Apply data augmentation to the training data
num_augmentations = 50
X_train_augmented, y_train_augmented = augment_data(X_train_reshaped, y_train, num_augmentations=num_augmentations)

# Reshape augmented data to match the model's input shape
X_train_augmented = X_train_augmented.reshape(-1, 1, X_train.shape[2])
print(X_train_augmented.shape)
# # Train the model using the augmented data
# model.fit(X_train_augmented, y_train_augmented, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

model = Sequential()
model.add(Flatten(input_shape=(1, 552))) # Flattening the input as it has only one timestep
model.add(Dense(128, activation='relu'))  # Adding another hidden layer with 64 units
model.add(Dropout(0.5))
model.add(Dense(21, activation='softmax')) # Output layer with 21 units for 21 classes

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train_augmented, y_train_augmented, epochs=50, batch_size=batch_size, validation_data=(X_test, y_test))


In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color=color)
line1, = ax1.plot(history.history['loss'], color=color, label='Train Loss')
line2, = ax1.plot(history.history['val_loss'], color=color, linestyle='dashed', label='Val Loss')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Accuracy', color=color)
line3, = ax2.plot(history.history['accuracy'], color=color, label='Train Acc')
line4, = ax2.plot(history.history['val_accuracy'], color=color, linestyle='dashed', label='Val Acc')
ax2.tick_params(axis='y', labelcolor=color)

# Added this line
lns = [line1, line2, line3, line4]
ax1.legend(handles=lns, loc='center right')

fig.tight_layout()
plt.show()
